# Data Cleaning

## Install Dependencies

In [1]:
%%capture
%pip install --upgrade xlrd
%pip install --upgrade openpyxl
%pip install --upgrade geopandas

## Import Modules

In [2]:
import os
import xlrd
import pandas as pd
import numpy as np
import geopandas as gpd
from datetime import datetime
from math import cos, asin, sqrt
from scipy.spatial import cKDTree
from shapely.geometry import MultiPoint

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

## Mount Drive

In [3]:
import os

team_name = 'capstone-power-grid-protagonists'
colab_path = f'/content/drive/Shareddrives/{team_name}/project'
studiolab_path = f'/home/studio-lab-user/sagemaker-studiolab-notebooks/{team_name}'

try:
    # Try to mount Google Drive and set project path
    from google.colab import drive
    drive.flush_and_unmount()
    drive.mount('/content/drive')
    print('')

    root_path = colab_path
    os.chdir(root_path)

except:
    try:
        # Try to set AWS SageMaker Studio Lab project path
        root_path = studiolab_path
        os.chdir(root_path)
    
    except:
        # Set current working directory as root path
        root_path = os.getcwd()
        os.chdir(root_path)

        # If the current folder is 'notebooks', move up one level
        if root_path.endswith('/notebooks'):
            root_path = '/'.join(root_path.split('/')[:-1])
            os.chdir(root_path)
        
print('Current working directory is:')
print(os.getcwd())

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive

Current working directory is:
/content/drive/Shareddrives/capstone-power-grid-protagonists/project


## Clean Data

In [4]:
def create_output_folder(folder, root_path=root_path):
    # If path does not exist, create it
    path = os.path.join(root_path, 'data', 'processed', folder)
    if os.path.isdir(path) == False: os.makedirs(path)

### EIA

#### EIA930 Interchange

In [5]:
# Set file to import
filepath = 'data/raw/eia/eia930_interchange_2022_jan_jun.csv'

# Set data types for import
dtypes = {'Balancing Authority': 'string',
          'Hour Number': 'int64',
          'Directly Interconnected Balancing Authority': 'string',
          'Interchange (MW)': 'object'}

# Set columns to parse dates on import
parse_dates = ['Data Date',
               'Local Time at End of Hour',
               'UTC Time at End of Hour']

eia930 = pd.read_csv(filepath, dtype=dtypes, parse_dates=parse_dates)

# Rename columns
eia930.rename(columns={'Balancing Authority': 'ba_code',
                       'Data Date': 'date',
                       'Hour Number': 'hour',
                       'Directly Interconnected Balancing Authority': 'connected_ba',
                       'Interchange (MW)': 'interchange_mw',
                       'Local Time at End of Hour': 'local_time',
                       'UTC Time at End of Hour': 'utc_time'}, inplace=True)

# Convert data types
eia930['interchange_mw'] = eia930['interchange_mw'].fillna(0) \
                                                   .replace(',','',regex=True) \
                                                   .astype('int64')

# Reorder columns
eia930 = eia930[['ba_code', 'connected_ba', 'interchange_mw',
                 'date', 'hour', 'local_time', 'utc_time']]

#### EIA930 Reference Tables

In [6]:
# Read multiple sheets from excel file
ba_ref = pd.read_excel('data/raw/eia/eia930_reference_tables.xlsx', sheet_name=['BAs', 'BA Connections'])

# Create a dataframe with balancing authorities
ba = ba_ref['BAs']
ba.columns = ba.columns.str.lower() \
                       .str.replace('[ |/]','_', regex=True) \
                       .str.replace('.','', regex=False)
ba = ba[ba['active_ba']=='Yes']
ba.drop(columns=['active_ba', 'activation_date', 'retirement_date'], inplace=True)

# Create a dataframe with balancing authority connections
ba_edges = ba_ref['BA Connections']
ba_edges.columns = ba_edges.columns.str.lower() \
                                   .str.replace('[ |/]','_', regex=True) \
                                   .str.replace('.','', regex=False)
ba_edges = ba_edges[ba_edges['active_connection']=='Yes']
ba_edges.drop(columns=['active_connection'], inplace=True)
ba_edges.rename(columns={'directly_interconnected_ba_code':'connected_ba'}, inplace=True)

# Add non-US connections manually
ba_edges = ba_edges.append({'ba_code': 'BCHA', 'connected_ba': 'AESO'}, ignore_index=True)
ba_edges = ba_edges.append({'ba_code': 'AESO', 'connected_ba': 'SPC'}, ignore_index=True)
ba_edges = ba_edges.append({'ba_code': 'SPC' , 'connected_ba': 'MHEB'}, ignore_index=True)
ba_edges = ba_edges.append({'ba_code': 'MHEB', 'connected_ba': 'IESO'}, ignore_index=True)
ba_edges = ba_edges.append({'ba_code': 'IESO', 'connected_ba': 'HQT'}, ignore_index=True)
ba_edges = ba_edges.append({'ba_code': 'HQT', 'connected_ba': 'NBSO'}, ignore_index=True)
ba_edges = ba_edges.append({'ba_code': 'CEN' , 'connected_ba': 'CFE'}, ignore_index=True)

ba_edges = ba_edges.groupby('ba_code')['connected_ba'] \
                   .apply('; '.join) \
                   .reset_index()

# Combine balancing authorities with connections
ba = ba.merge(ba_edges, on='ba_code', how='outer')

### EPA

#### eGRID

In [7]:
df_list = []

for name in os.listdir('data/raw/epa'):
    year = name.split('_')[0][-4:]

    # Sheet specific to power plants
    sheet = 'PLNT' + year[-2:] 

    # Formatting was different in earlier years, so the header starts in a different spot 
    header = 1 if int(year) > 2012 else 4 

    df = pd.read_excel('data/raw/epa/'+name, sheet_name=sheet, header=header)
    
    # Not all years include this column, so manually setting it
    df['YEAR'] = int(year)

    df_list.append(df)

# Combine the dataframes
egrid = pd.concat(df_list)

# Convert column names to lowercase
egrid.columns = egrid.columns.str.lower()

# Drop columns based on percentage of NaN values
perc = 60.0
min_count = int(((100-perc)/100)*egrid.shape[0]+1)

egrid_col_before = egrid.columns.tolist()
egrid.dropna(axis=1, thresh=min_count, inplace=True)
egrid_col_after = egrid.columns.tolist()

# Clean up singular value
egrid['plhtrt'] = egrid['plhtrt'].replace('N/A09.2664',0, regex=True) \
                                 .fillna(0) \
                                 .astype('float64')
                                 
# Set data types for columns with codes/IDs
egrid[['orispl', 'oprcode', 'utlsrvid', 'fipscnty']] = \
    egrid[['orispl', 'oprcode', 'utlsrvid', 'fipscnty']].fillna(0).astype('int64')

egrid['fipscnty'] = egrid['fipscnty'].astype(str).str.pad(width=3, side='left', fillchar='0')
egrid['fipscnty'] = egrid['fipsst'].astype(str) + egrid['fipscnty']

# Sort the dataframe
egrid.sort_values(by=['year', 'pstatabb', 'pname'],
                  ascending=[False, True, True],
                  ignore_index=True,
                  inplace=True)

In [8]:
# Show columns that were dropped based on the percentage of NaN values
dropped_cols = [col for col in egrid_col_before if col not in egrid_col_after]
str(dropped_cols)

"['seqplt20', 'coalflag', 'nbfactor', 'rmbmflag', 'chpflag', 'usethrmo', 'pwrtoht', 'elcalloc', 'psflag', 'plhgan', 'plhgrta', 'plch4ra', 'pln2ora', 'plc2era', 'plhgra', 'plc2ecrt', 'plhgcrt', 'unhg', 'unnoxsrc', 'unnozsrc', 'unso2src', 'unco2src', 'unch4src', 'unn2osrc', 'bionox', 'bionoxoz', 'bioso2', 'bioco2', 'bioch4', 'bion2o', 'chpchti', 'chpchtioz', 'chpnox', 'chpnoxoz', 'chpso2', 'chpco2', 'chpch4', 'chpn2o', 'seqplt19', 'seqplt18', 'seqplt16', 'seqplt12', 'frsid', 'opprnum', 'opprname', 'pcaname', 'pcaid', 'ccflag', 'combust', 'sourcem', 'plpfgnct', 'ownrnm01', 'ownruc01', 'ownrpr01', 'ownrnm02', 'ownruc02', 'ownrpr02', 'ownrnm03', 'ownruc03', 'ownrpr03', 'ownrnm04', 'ownruc04', 'ownrpr04', 'ownrnm05', 'ownruc05', 'ownrpr05', 'ownrnm06', 'ownruc06', 'ownrpr06', 'ownrnm07', 'ownruc07', 'ownrpr07', 'ownrnm08', 'ownruc08', 'ownrpr08', 'ownrnm09', 'ownruc09', 'ownrpr09', 'ownrnm10', 'ownruc10', 'ownrpr10', 'ownrnm11', 'ownruc11', 'ownrpr11', 'ownrnm12', 'ownruc12', 'ownrpr12', 'ow

### HIFLD

In [9]:
# Create function for common cleaning activities
def clean_hifld_data(df):   
    # Set NaN values
    df.replace('NOT AVAILABLE', np.NaN, inplace=True)
    df.replace(-999999, np.NaN, inplace=True)

    # Convert date columns
    df['SOURCEDATE'] = pd.to_datetime(df['SOURCEDATE'])
    df['VAL_DATE'] = pd.to_datetime(df['VAL_DATE'])

    # Convert column names to lowercase
    df.columns = df.columns.str.lower()

    # Rename columns
    df.rename(columns={'latitude':'lat',
                       'longitude':'lon',
                       'sourc_long':'source_lon'}, inplace=True)

    # Drop columns
    cols = ['x', 'y', 'objectid']
    for col in cols:
        if col in df.columns.tolist():
            df.drop(columns=col, inplace=True)
    
    # Set data type for columns with codes/IDs
    cols = ['zip', 'countyfips', 'naics_code']
    for col in cols:
        if col in df.columns.tolist():
            df[col] = df[col].fillna(0).astype('str')

    # Update text to leverage title casing
    cols = ['name', 'address', 'city', 'county', 'type', 'naics_desc', 'operator',
            'owner', 'sub_1', 'sub_2', 'status', 'val_method', 'volt_class']
    for col in cols:
        if col in df.columns.tolist():
            df[col] = df[col].str.title()

    return df

In [10]:
# Create functions to find closest distance between latitude and longitude
# https://stackoverflow.com/questions/41336756/find-the-closest-latitude-and-longitude

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    hav = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(hav))

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))

# Create function to get the substation id. If there are multiple matches,
# return id of the closest substation.
def get_sub_code(sub_df, sub_name, src_lat, src_lon):
    matches = sub_df[sub_df['name'] == sub_name]
    if len(matches) == 0:
        return np.NaN
    elif len(matches) == 1:
        return matches['sub_code'].values[0]
    else:
        dst = matches[['sub_code', 'lat', 'lon']].to_dict('records')
        src = {'lat': src_lat, 'lon': src_lon}
        return closest(dst, src)['sub_code']

# Create function to return the closest substation based on lat and lon
# https://gis.stackexchange.com/questions/222315/finding-nearest-point-in-other-geodataframe-using-geopandas
def ckdnearest(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda g: (g.coords.xy[1][-1],
                                                     g.coords.xy[0][-1]))))
    nB = np.array(list(gdA.geometry.apply(lambda g: (g.coords.xy[1][ 0],
                                                     g.coords.xy[0][ 0]))))
    nC = np.array(list(zip(gdB['lat'], gdB['lon'])))

    btree = cKDTree(nC)

    distA, idxA = btree.query(nA, k=1)
    distB, idxB = btree.query(nB, k=1)

    gdB_nearestA = gdB.iloc[idxA][['sub_code', 'name']] \
                      .reset_index(drop=True) \
                      .rename(columns={'sub_code':'sub_code_1',
                                       'name':'sub_name_1'})   
    gdB_nearestB = gdB.iloc[idxB][['sub_code', 'name']] \
                      .reset_index(drop=True) \
                      .rename(columns={'sub_code':'sub_code_2',
                                       'name':'sub_name_2'})

    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearestA.astype(str),
            round(pd.Series(distA, name='sub_dist_1'), 3),
            gdB_nearestB.astype(str),
            round(pd.Series(distB, name='sub_dist_2'), 3)
        ],
        axis=1)

    return gdf

##### Electric Substations

In [11]:
# Import raw data and do initial cleaning
path = 'data/raw/hifld/electric_substations.csv'
subs = pd.read_csv(path, low_memory=False)
subs = clean_hifld_data(subs)

# Remove substations that are not in service
subs = subs[(subs['status'].isnull()) | (subs['status']=='In Service')]

# Remove substations not in the United States
subs.drop(subs[subs['country']!='USA'].index, inplace=True)

# Rename id column for consistency with power plant code formatting
subs.rename(columns={'id':'sub_code'}, inplace=True)

# Convert substation codes to strings
subs['sub_code'] = subs['sub_code'].astype(str)

In [12]:
# Get substations with duplicate name
subs_dup_name = subs[subs.duplicated('name')]['name']

# Get percentage of duplicates
pct = len(subs_dup_name)/len(subs)
print('Percentage of substations with duplicate names:')
print("{:.2%}".format(pct))

Percentage of substations with duplicate names:
4.43%


##### Power Plants

In [13]:
# Import raw data and do initial cleaning
path = 'data/raw/hifld/power_plants.csv'
plants = pd.read_csv(path, low_memory=False)
plants = clean_hifld_data(plants)

In [14]:
# Combine substations (sub_1 and sub_2) in to a single column
plants['subs'] = plants['sub_1']

mask = (plants['sub_2'].notnull())
plants.loc[mask, 'subs'] = plants[mask].apply(lambda x: [x['sub_1'], x['sub_2']], axis=1)

plants.drop(columns=['sub_1', 'sub_2'], inplace=True)
plants = plants.explode('subs').reset_index()

total_subs = len(plants[plants['subs'].notnull()])

In [15]:
# Get substations with duplicate name
plants_dup_sub = plants[plants['subs'].isin(subs_dup_name)]

# Get percentage of duplicates
pct = len(plants_dup_sub)/len(plants[plants['subs'].notnull()])

print('Percentage of linked substations with duplicate names:')
print("{:.2%}".format(pct))

Percentage of linked substations with duplicate names:
16.49%


In [16]:
# Clean up known spelling errors for substations
plants['subs'].replace('Birdslanding', 'Birds Landing', inplace=True)
plants['subs'].replace('Harbor (Substatoin Q)', 'Station Q (Harbor)', inplace=True)
plants.loc[plants['plant_code']=='56656', ['subs']] = str.title("SANTA ROSA TOP O' DA HILL")
plants.loc[plants['plant_code']=='358', ['subs']] = ''
plants.loc[plants['plant_code']=='57192', ['subs']] = ''

# Merge the substation code for those that have a unique name
unique_subs = subs.drop_duplicates(subset='name', keep=False)
plants['connected_sub'] = plants.merge(unique_subs,
                                    how='left',
                                    left_on='subs',
                                    right_on='name')['sub_code']

# Get substation ID based on locations for those that had duplicate names
mask = plants['subs'].notnull() & plants['connected_sub'].isnull()
plants.loc[mask, 'connected_sub'] = plants[mask].apply(
    lambda x: get_sub_code(subs, x['subs'], x['lat'], x['lon']), axis=1)

# If the closest substation happens to be one already found for that power
# plant, remove it
plants.drop_duplicates(subset=['plant_code', 'connected_sub'], inplace=True)

In [17]:
# Identify the number of substations where a substation code was not found
found_subs = len(plants[plants['connected_sub'].notnull()])
missing_subs = total_subs - found_subs
missing_pct = "{:.2%}".format(missing_subs/total_subs)

print(f'There are {missing_subs} substations where a substation code was not found')
print(f'This represents {missing_pct} of the substations in this dataset')

There are 20 substations where a substation code was not found
This represents 0.56% of the substations in this dataset


In [18]:
# Create a mapping between plant codes and substation codes
plants['connected_sub'] = plants['connected_sub'].fillna(0) \
                                                 .astype(int) \
                                                 .astype(str) \
                                                 .replace('0', '')

linked_subs = plants.groupby('plant_code')['connected_sub'] \
                    .apply('; '.join) \
                    .reset_index() \
                    .replace('', np.NaN) \
                    .replace(';$', '', regex=True)

# Merge power plants with substation code mapping
plants = plants[plants.columns[:-1]].drop_duplicates('plant_code') \
                                    .merge(linked_subs, how='left', on='plant_code') \
                                    .drop(columns=['index', 'subs'])

In [19]:
# Create a mapping between plant codes and balancing authority codes
linked_bas = egrid[['orispl', 'bacode']].astype(str) \
                                        .replace('nan', np.NaN) \
                                        .dropna() \
                                        .drop_duplicates() \
                                        .groupby('orispl')['bacode'] \
                                        .apply('; '.join) \
                                        .reset_index() \
                                        .rename(columns={'bacode':'connected_ba'})

# Merge power plants with balancing authority code mapping
plants = plants.merge(linked_bas[linked_bas['connected_ba'].isin(ba.ba_code.tolist())],
                      how='left',
                      left_on='plant_code',
                      right_on='orispl') \
               .drop(columns=['orispl'])

In [20]:
# Get the lat/lon values for each balancing authority, based on the central
# location of all connected power plants
# https://gis.stackexchange.com/questions/29349/how-should-i-go-about-calculating-the-centroid-of-several-lat-long-points-in-pyt
bas_pos = plants.copy()[['plant_code', 'lat', 'lon', 'connected_ba']].dropna()
bas_pos['connected_ba'] = bas_pos['connected_ba'].str.split('; ')
bas_pos = bas_pos.explode('connected_ba')

bas_pos['coords'] = list(zip(bas_pos['lat'], bas_pos['lon']))
bas_pos = bas_pos.groupby('connected_ba')['coords'] \
                 .apply(list) \
                 .reset_index() \
                 .rename(columns={'connected_ba':'ba_code'})

bas_pos['lat'] = bas_pos['coords'].apply(lambda x: MultiPoint(x).centroid.x)
bas_pos['lon'] = bas_pos['coords'].apply(lambda x: MultiPoint(x).centroid.y)

bas_pos.drop(columns='coords', inplace=True)

# Set lat/lon values manually as necessary
bas_pos = bas_pos.append({'ba_code': 'BCHA', 'lat': '50.0000000', 'lon': '-122.5000000'}, ignore_index=True)
bas_pos = bas_pos.append({'ba_code': 'AESO', 'lat': '50.0000000', 'lon': '-113.9904714'}, ignore_index=True)
bas_pos = bas_pos.append({'ba_code': 'SPC',  'lat': '50.0000000', 'lon': '-103.8070100'}, ignore_index=True)
bas_pos = bas_pos.append({'ba_code': 'MHEB', 'lat': '50.0000000', 'lon': '-96.50000000'}, ignore_index=True)

bas_pos = bas_pos.append({'ba_code': 'IESO', 'lat': '44.5000000', 'lon': '-78.50000000'}, ignore_index=True)
bas_pos = bas_pos.append({'ba_code': 'HQT',  'lat': '46.0000000', 'lon': '-75.14008212'}, ignore_index=True)
bas_pos = bas_pos.append({'ba_code': 'NBSO', 'lat': '46.0000000', 'lon': '-71.50000000'}, ignore_index=True)

bas_pos = bas_pos.append({'ba_code': 'GLHB', 'lat': '38.0272948', 'lon': '-88.44280000'}, ignore_index=True)
bas_pos = bas_pos.append({'ba_code': 'GRID', 'lat': '38.4726256', 'lon': '-110.0000000'}, ignore_index=True)
bas_pos = bas_pos.append({'ba_code': 'CEN',  'lat': '27.5631090', 'lon': '-101.5720000'}, ignore_index=True)
bas_pos = bas_pos.append({'ba_code': 'CFE',  'lat': '27.5631090', 'lon': '-105.0000000'}, ignore_index=True)

# Merge lat/lon data with balancing authorities
eia930 = eia930.merge(bas_pos, how='outer', on='ba_code')
ba = ba.merge(bas_pos, how='outer', on='ba_code')

#### Electric Power Transmission Lines

In [21]:
# Import raw data and do initial cleaning
path = 'data/raw/hifld/electric_power_transmission_lines.geojson'
lines = gpd.read_file(path).explode(index_parts=False)
lines = clean_hifld_data(pd.DataFrame(lines))

# Rename id column for consistency with power plant code formatting
lines.rename(columns={'id':'line_code'}, inplace=True)

In [22]:
# As this dataset is used to connect substations, drop rows where substations
# are not included to speed up processing
lines.dropna(subset=['sub_1', 'sub_2'], inplace=True)

# Remove substations that are not in service
lines = lines[(lines['status'].isnull()) | (lines['status']=='In Service')]

# As the substation names are not unique, get the nearest substations based
# on latitude and longitude values
lines = ckdnearest(lines, subs)

# Create a mask when the closest substations match the names provided and when
# both substations are not the same
mask1 = ((lines['sub_name_1']==lines['sub_1']) & (lines['sub_name_2']==lines['sub_2']))
mask2 = ((lines['sub_name_1']==lines['sub_2']) & (lines['sub_name_2']==lines['sub_1']))
mask3 = (lines['sub_code_1']!=lines['sub_code_2'])
matches = (mask1 | mask2) & mask3

# Get the maximum distance that a match was found from the lat/lon values, times 2.
# This will be used as a threshold for guessing at other matches.
max_dist = lines[matches][['sub_dist_1', 'sub_dist_2']].max(axis=1).max(axis=0)*2

# Create a mask for educated guesses based on lat/lon
mask4 = ((lines['sub_dist_1']<=max_dist) & (lines['sub_dist_2']<=max_dist))
guesses = (~matches & mask3 & mask4)

In [23]:
# Identify the number of lines where substations were not matched
matched_lines = len(lines[matches])
guessed_lines = len(lines[guesses])
total_lines = len(lines)
missing_lines = total_lines - (matched_lines + guessed_lines)
missing_pct = "{:.2%}".format(missing_lines/total_lines)

print(f'There are {missing_lines} lines where a substation was not matched')
print(f'This represents {missing_pct} of the lines in this dataset')

There are 1156 lines where a substation was not matched
This represents 1.29% of the lines in this dataset


In [24]:
# Filter dataframe by both matches and guesses and drop any duplicates
lines = lines[matches|guesses]

# Join substations in to single column
lines['connected_sub'] = lines.apply(lambda x: '; '.join([x['sub_code_1'], x['sub_code_2']]), axis=1)

# Clean up dataframe
lines = lines.drop_duplicates('line_code') \
             .drop(columns=['sub_1',
                            'sub_2',
                            'shape__length',
                            'geometry',
                            'sub_code_1',
                            'sub_name_1',
                            'sub_dist_1',
                            'sub_code_2',
                            'sub_name_2',
                            'sub_dist_2',])

# Create mapping of just substation codes
linked_subs = pd.DataFrame()
linked_subs[['sub_code', 'connected_sub']] = lines['connected_sub'].str.split('; ', expand=True)
linked_subs = linked_subs.dropna().groupby('sub_code')['connected_sub'] \
                         .apply('; '.join) \
                         .reset_index()

# Merge substations with substation code mapping
subs = subs.merge(linked_subs, how='left', on='sub_code')

### DOE

#### Electric Disturbance Events

In [25]:
df_list = []

# Set path for files
path = 'data/raw/doe/'

# Set known NaN values
nan = ['Unknown','Unknown ','Ongoing','None','UNK','UNK ','nan']

# Set dates/times to strings due to mixed formatting in different years
dtypes = {'Date Event Began': 'string',
          'Time Event Began': 'string',
          'Date of Restoration': 'string',
          'Time of Restoration': 'string'}

# Get all files in source dir after 2010
for name in os.listdir(path):
    if name.endswith('annual_summary.xls'):
        year = int(name.split('_')[0])
        if year > 2010:
            # The workbook is opened first due to a known issue with warnings about
            # file size: https://github.com/pandas-dev/pandas/issues/16620
            wb = xlrd.open_workbook(path+name, logfile=open(os.devnull, 'w'))
            df = pd.read_excel(wb, na_values=nan, dtype=dtypes, header=1)
            df_list.append(df)

# Combine the dataframes
disturb = pd.concat(df_list)

# Remove rows that are mostly NaN values due to Excel formatting
disturb.dropna(thresh=5, inplace=True)
disturb.reset_index(drop=True, inplace=True)

# Remove month column that is not present in older data sets
disturb.drop(columns='Month', inplace=True)

# Rename columns
disturb.columns = ['date_start', 'time_start', 'date_end', 'time_end',
                   'areas_affected', 'nerc_region', 'alert_criteria',
                   'event_type', 'demand_loss_mw', 'num_customers_affected']

# Filter out headings that were included multiple times in data set
disturb = disturb[disturb['date_start']!='Date Event Began']

# Convert date/time columns to datetime type
cols = ['date_start','date_end','time_start','time_end']
disturb[cols] = disturb[cols].apply(pd.to_datetime)

# Fill empty and future values (which appear to be a mistake) with 12/01/1900
fake_date = pd.to_datetime('1900-12-01 00:00')
disturb['date_end'].fillna(fake_date, inplace=True)
disturb['time_end'].fillna(fake_date, inplace=True)
disturb.loc[disturb['date_end'] > datetime.today(), 'date_end'] = fake_date

# Combine date and time data in to a single column
disturb['date_start'] = pd.to_datetime(
    disturb['date_start'].apply(lambda x: x.strftime('%Y-%m-%d')) + ' ' + \
    disturb['time_start'].apply(lambda x: x.strftime('%H:%M:%S')))

disturb['date_end'] = pd.to_datetime(
    disturb['date_end'].apply(lambda x: x.strftime('%Y-%m-%d')) + ' ' + \
    disturb['time_end'].apply(lambda x: x.strftime('%H:%M:%S')))

# Create ID for events before exploding them
disturb.sort_values('date_start', ascending=False, inplace=True)
disturb.reset_index(drop=True, inplace=True)
disturb.reset_index(inplace=True)
disturb.rename(columns={'index':'event_id'}, inplace=True)

# Move column names in to variables to make code more compact
aa = 'areas_affected'
sa = 'state_affected'

# Import states and abbreviations to assist with extration in data set
states = pd.read_csv(path + 'states.csv', header=None, names=['name', 'abbr'])
states = dict(states.values)

# Due to inconsitent formatting, check for states that are separated by
# other punctuation other than ";", or by "and".
for s1 in states:
    for s2 in states:
        disturb[aa].replace(fr'{s1}([\W]+| and ){s2}(?! County)',
                            s1+';'+s2,
                            regex=True,
                            inplace=True)

# Split events that affected multiple states on to separate lines
disturb[aa] = disturb[aa].str.replace(';$', '', regex=True)
disturb[aa] = disturb[aa].str.split(';')
disturb = disturb.explode(aa)

# Separate states and areas in to separate columns
disturb[aa] = disturb[aa].str.replace(':$', '', regex=True).astype(str)
disturb[sa] = disturb[aa].str.split(':', expand=True)[0]
disturb[aa] = disturb[aa].str.split(':', expand=True)[1]

# Some older data put the cities/counties before the state, separated by a
# comma. For empty values in the area affected, grab all text aside from the
# final split. For the state, that final split.
disturb[aa].fillna(
    disturb[sa].str.split(',').apply(lambda x: ','.join(x[:-1])),
    inplace=True)
disturb[sa] = disturb[sa].str.split(',').apply(lambda x: x[-1])

# Some data includes the general location in a state, but that is not useful
# when joining with other data, so it is being removed
pat = ''.join(['( and )*',
               '(Central|Western|Eastern|Northern|Southern|',
               'Northeast|Northwest|Southeast|Southwest|Upstate)'])

disturb[sa].replace(pat, '', regex=True, inplace=True)

# Clean up a few entries that still contain counties in the states column
disturb = disturb[disturb[sa].str.contains('County')==False]

# Clean up entried for D.C. to match state data for abbreviations
disturb[sa].replace('District of Columbia|D.C.',
                    'District Of Columbia',
                    regex=True,
                    inplace=True)

# Create a function to grab state abbreviations
def abbreviate_state(x):
    state = x.strip()
    state_guess = x.split(' ')[-1].strip()
    if state in states.keys():
        return states[state]
    elif state_guess in states.keys():
        return states[state_guess]
    else:
        return None

disturb[sa] = disturb[sa].apply(lambda x: abbreviate_state(x))

# Drop rows where a state was not able to be identified
disturb.dropna(subset=[sa], inplace=True)

# Expode counties/cities/areas on to individual rows
disturb[aa] = disturb[aa].str.split(',')
disturb = disturb.explode(aa)

# Clean up leading/trailing spaces and non-alpha characters
disturb[aa] = disturb[aa].str.strip()
disturb[aa] = disturb[aa].replace('[^a-zA-Z ]', '', regex=True)

# Clean up a couple of entries that start with "and"
disturb[aa].replace('^and ', '', regex=True, inplace=True)

# Clean up areas that include "area" pr "metro" to make them easier to match
disturb[aa].replace('[Aa]rea$', '', regex=True, inplace=True)
disturb[aa].replace('[Mm]etro(politan| |$)|', '', regex=True, inplace=True)
disturb[aa] = disturb[aa].str.strip()

# Clean up spelling
disturb[aa].replace('Substaion', 'Substation', regex=True, inplace=True)

# Remove stray entries ending in "of"
disturb = disturb[disturb[aa].str.endswith(' of')==False]

# Make all event types lowercase
disturb['event_type'] = disturb['event_type'].str.lower()

# Reduce the number of event tyes from 151 to 7 by categorizing them
disturb['event_type'] = disturb['event_type'].str.replace(
    '(.*)(weather|storm|disaster|earthquake)(.*)', 'severe_weather', regex=True)
disturb['event_type'] = disturb['event_type'].str.replace(
    '(.*)(vandalism|vandalsim|cyber|attack|suspicious|sabotage)(.*)', 'vandalism/cyber_attack', regex=True)
disturb['event_type'] = disturb['event_type'].str.replace(
    '(.*)(system|transmission|interruption)(.*)', 'system_operations', regex=True)
disturb['event_type'] = disturb['event_type'].str.replace(
    '(.*)(appeal)(.*)', 'public_appeal', regex=True)
disturb['event_type'] = disturb['event_type'].str.replace(
    '(.*)(supply|inadequacy|loss)(.*)', 'fuel_supply_emergency', regex=True)
disturb['event_type'] = disturb['event_type'].str.replace(
    '(.*)(malfunction|shed|failure|equipment|fault|islanding)(.*)', 'malfunction/equipment_failure', regex=True)
disturb['event_type'] = disturb['event_type'].str.replace(
    '(.*)(event)(.*)', 'other', regex=True)

# Clean up some city names to use county instead
disturb['areas_affected'].replace('Tacoma', 'Pierce County', inplace=True)
disturb['areas_affected'].replace('Houston', 'Harris County', inplace=True)
disturb['areas_affected'].replace('Newark', 'Essex County', inplace=True)
disturb['areas_affected'].replace('Salt Lake City', 'Salt Lake County', inplace=True)
disturb['areas_affected'].replace('Portland', 'Multnomah County', inplace=True)

# Import FIPS data to incorporate for counties
xlsx = path + 'counties.xlsx'
counties = pd.read_excel(xlsx, header=4)
counties.columns = ['level', 'state_fips', 'county_fips',
                    'subdiv_fips', 'place_fips', 'city_fips', 'name']
state_fips = dict(counties[counties['level']==40][['state_fips', 'name']].values)
counties['state'] = counties['state_fips'].apply(lambda x: state_fips[x] if x in state_fips.keys() else np.NaN)
counties['state'] = counties['state'].apply(lambda x: states[x] if x in states.keys() else np.NaN)
counties['county_fips'] = counties['county_fips'].astype(str).str.pad(width=3, side='left', fillchar='0')
counties['county_fips'] = counties['state_fips'].astype(str) + counties['county_fips']
counties = counties[counties['level']==50].dropna()[['county_fips', 'name', 'state']]

# Merge disturbances with FIPS data
disturb = disturb.merge(counties,
                        how='left',
                        left_on=['state_affected', 'areas_affected'],
                        right_on=['state', 'name'])

# Clear areas where a county was not found
disturb.loc[disturb['county_fips'].isna(), 'areas_affected'] = np.NaN
disturb.rename(columns={'areas_affected':'county_affected',
                        'county_fips':'county_affected_fips'}, inplace=True)

# Drop and reorder columns
disturb = disturb[['event_id',
                   'date_start',
                   'date_end',
                   'state_affected',
                   'county_affected',
                   'county_affected_fips',
                   'nerc_region',
                   'alert_criteria',
                   'event_type',
                   'demand_loss_mw',
                   'num_customers_affected']]
disturb.reset_index(drop=True, inplace=True)

### Enriched Output

In [26]:
eia930.sample(3)

,ba_code,connected_ba,interchange_mw,date,hour,local_time,utc_time,lat,lon
761612,PACE,IPCO,201.0,2022-06-14,11.0,2022-06-14 11:00:00,2022-06-14 17:00:00,40.806257,-111.00161
1192073,TVA,PJM,774.0,2022-03-05,21.0,2022-03-05 21:00:00,2022-03-06 03:00:00,35.374538,-86.585067
1114834,SWPP,AECI,827.0,2022-06-25,9.0,2022-06-25 09:00:00,2022-06-25 14:00:00,37.772794,-98.070741


In [27]:
ba.sample(3)

,ba_code,ba_name,time_zone,region_country_code,region_country_name,generation_only_ba,demand_by_ba_subregion,us_ba,connected_ba,lat,lon
1,AECI,"Associated Electric Cooperative, Inc.",Central,MIDW,Midwest,No,No,Yes,MISO; SPA; SWPP; TVA,39.267224,-92.881623
11,DEAA,"Arlington Valley, LLC",Arizona,SW,Southwest,Yes,No,Yes,SRP,33.3417,-112.8897
24,GWA,"NaturEner Power Watch, LLC",Mountain,NW,Northwest,Yes,No,Yes,NWMT,48.519898,-112.146067


In [28]:
egrid.sample(3)

,year,pstatabb,pname,orispl,oprname,oprcode,utlsrvnm,utlsrvid,sector,baname,...,plsopr,plgtpr,plofpr,ploppr,pltnpr,pltrpr,plthpr,plcypr,plcnpr,numblr
53824,2012,AK,Stebbins,57055,Alaska Village Elec Coop Inc,221,Alaska Village Elec Coop Inc,221,NaN,NaN,...,0.0,0.0,0.00000,0.0,100.0,0.0,0.0,100.0,0.0,0.0
56325,2012,IL,Grand Tower,862,Ameren Energy Generating Co,520,Ameren Illinois Company,56697,NaN,NaN,...,0.0,0.0,0.00000,0.0,100.0,0.0,0.0,100.0,0.0,2.0
4325,2020,IN,US Steel Corp - Gary Works,50733,Northern Indiana Pub Serv Co,13756,United States Steel-Gary,19526,Industrial CHP,Midcontinent Independent Transmission System O...,...,0.0,0.0,0.55213,0.0,1.0,0.0,0.0,1.0,0.0,NaN


In [29]:
subs.sample(3)

,sub_code,name,city,state,zip,type,status,county,countyfips,country,...,source,sourcedate,val_method,val_date,lines,max_volt,min_volt,max_infer,min_infer,connected_sub
50270,168600,Unknown168600,Pocola,OK,74902,Substation,In Service,Le Flore,40079,USA,...,IMAGERY,2019-12-10 00:00:00+00:00,Imagery,2019-12-10 00:00:00+00:00,1.0,69.0,69.0,Y,Y,168601
32084,146818,Unknown146818,Conway,SC,29526,Substation,In Service,Horry,45051,USA,...,IMAGERY,2016-11-10 00:00:00+00:00,Imagery,2016-11-10 00:00:00+00:00,1.0,115.0,115.0,Y,Y,146827
57857,201301,Unknown201301,NaN,ID,83314,Substation,In Service,Twin Falls,16083,USA,...,IMAGERY,2014-12-05 00:00:00+00:00,Imagery,2014-12-05 00:00:00+00:00,3.0,138.0,138.0,Y,Y,201567; 204861


In [30]:
plants.sample(3)

,plant_code,name,address,city,state,zip,telephone,type,status,county,...,coal_used,ngas_used,oil_used,net_gen,cap_factor,lines,source_lat,source_lon,connected_sub,connected_ba
10682,62165,Snipesville,NaN,Denton,GA,31532,NaN,Solar Photovoltaic,L,Jeff Davis,...,0.0,0.0,0.0,0.0,0.000000,0,31.715814,-82.74946,NaN,SOCO
8565,59944,"Conetoe Ii Solar, Llc",189 Leigh Road,Conetoe,NC,27819,NaN,Solar Photovoltaic,Op,Edgecombe,...,0.0,0.0,0.0,156601.0,0.223460,0,35.822000,-77.48100,NaN,PJM
1920,3799,Low Moor,9319 Rich Patch Road,Covington,VA,24426,(888) 216-3718,Petroleum Liquids,Op,Alleghany,...,0.0,0.0,1786.0,599.0,0.001425,1,37.777000,-79.89200,146407,PJM


In [31]:
lines.sample(3)

,line_code,type,status,naics_code,naics_desc,source,sourcedate,val_method,val_date,owner,voltage,volt_class,inferred,connected_sub
51836,168641,Ac; Overhead,In Service,221121,Electric Bulk Power Transmission And Control,"IMAGERY, https://www.energy.gov/sites/prod/fil...",2019-10-22 00:00:00+00:00,Imagery,2019-11-06 00:00:00+00:00,NaN,69.0,Under 100,Y,167626; 167625
63521,178263,Ac; Overhead,NaN,221121,Electric Bulk Power Transmission And Control,"IMAGERY, https://www.energy.gov/sites/prod/fil...",2021-03-21 00:00:00+00:00,Imagery,2021-03-21 00:00:00+00:00,NaN,69.0,Under 100,Y,159717; 175872
80421,301935,Ac; Overhead,NaN,221121,Electric Bulk Power Transmission And Control,Open Access Technology International,2017-01-01 00:00:00+00:00,Imagery/Other,2017-01-01 00:00:00+00:00,Arizona Public Service Company,69.0,Under 100,N,301413; 303257


In [32]:
disturb.sample(3)

,event_id,date_start,date_end,state_affected,county_affected,county_affected_fips,nerc_region,alert_criteria,event_type,demand_loss_mw,num_customers_affected
805,419,2021-01-26 20:49:00,2021-01-28 21:14:00,CA,NaN,NaN,WECC,"Loss of electric service to more than 50,000 c...",severe_weather,841.0,255715.0
1623,795,2020-02-07 16:25:00,2020-02-08 12:00:00,VT,NaN,NaN,NPCC,"Loss of electric service to more than 50,000 c...",severe_weather,NaN,123359.0
502,225,2021-07-05 14:04:00,2021-07-05 14:41:00,OH,NaN,NaN,RF/SERC,Unplanned evacuation from its Bulk Electric Sy...,system_operations,0.0,0.0


In [33]:
# Output EIA data
create_output_folder('eia')
eia930.to_csv('data/processed/eia/eia930_interchange.csv', index=False)
ba.to_csv('data/processed/eia/balancing_authorities.csv', index=False)

# Output EPA data
create_output_folder('epa')
egrid.to_csv('data/processed/epa/egrid.csv', index=False)

# Output HIFLD data
create_output_folder('hifld')
subs.to_csv('data/processed/hifld/substations.csv', index=False)
plants.to_csv('data/processed/hifld/power_plants.csv', index=False)
lines.to_csv('data/processed/hifld/power_lines.csv', index=False)

# Output DOE data
create_output_folder('doe')
disturb.to_csv('data/processed/doe/disturbances.csv', index=False)